# 🌌 Spectral Affinity Master: Neural Chromatic Hybrid (v5.0)

This system unifies **DSP-based Harmonic Mixing**, **Neural Semantic Affinity (MERT)**, and **Professional Spectral Restoration**.

### 🧬 Professional Hybrid & Restoration Features:
- 🧠 **Neural Brain (MERT):** Extracts deep sonic signatures to understand the 'vibe' and group affinity.
- 🔥 **DSP Brain (nnAudio GPU):** Precision detection of Musical Key, Mode, Energy, and spectral intensity.
- 🌈 **Chromatic Flow Engine:** Intelligent sequencing following the **Camelot Wheel** (e.g., 1B → 2B or 1B → 1A).
- 🏖️ **Island Clustering:** Groups tracks into semantic 'Islands' based on group affinity before sequencing.
- 💎 **Spectre Restoration:** AI-driven harmonic synthesis to restore MP3 high-end losses (Upsampling to 48kHz + Saturation + FLAC export).
- 📦 **Turbo-Batching:** Full CUDA saturation for lightning-fast analysis of large libraries.

### 1. 🛠️ High-Performance Environment

In [ ]:
import os, warnings, json, torch, torchaudio, librosa, glob, shutil, re, gc, pathlib
import numpy as np
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from IPython.display import HTML, FileLink, display
import torchaudio.transforms as T
import torchaudio.functional as F
from transformers import Wav2Vec2FeatureExtractor, AutoModel, logging as hf_logging
from sklearn.preprocessing import normalize
from sklearn.cluster import AffinityPropagation, KMeans as skKMeans

warnings.filterwarnings('ignore')
hf_logging.set_verbosity_error()
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔥 ACCELERATOR: {torch.cuda.get_device_name(0) if device == 'cuda' else 'CPU'}")

try:
    from nnAudio.Spectrogram import CQT1992v2
except:
    !pip install -q nnAudio transformers
    from nnAudio.Spectrogram import CQT1992v2

try:
    import cuml
    from cuml.cluster import KMeans as cuKMeans
    HAS_CUML = True
except:
    HAS_CUML = False

### 2. 🧠 The Unified Neural & DSP Core

In [ ]:
class SpectralMasterEngine:
    def __init__(self, device='cuda', sr=22050, cache_file='spectral_master_cache.json'):
        self.device = device
        self.sr = sr
        self.cache_file = cache_file
        self.cache = self._load_cache()
        
        # 1. DSP Layers
        self.cqt_layer = CQT1992v2(sr=self.sr, n_bins=84, bins_per_octave=12).to(self.device)
        major = torch.tensor([6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88], device=device)
        minor = torch.tensor([6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17], device=device)
        self.profiles = torch.stack([torch.roll(major, i) for i in range(12)] + 
                                    [torch.roll(minor, i) for i in range(12)]).t()
        
        # 2. Neural Layers (MERT)
        print("🧠 Loading MERT Neural Brain...")
        self.mert_model_id = "m-a-p/MERT-v1-95M"
        self.processor = Wav2Vec2FeatureExtractor.from_pretrained(self.mert_model_id, trust_remote_code=True)
        self.mert_model = AutoModel.from_pretrained(self.mert_model_id, trust_remote_code=True).to(self.device)
        self.mert_model.eval()

    def _load_cache(self):
        if os.path.exists(self.cache_file):
            with open(self.cache_file, 'r') as f: return json.load(f)
        return {}

    def save_cache(self):
        with open(self.cache_file, 'w') as f: json.dump(self.cache, f)

    def get_camelot(self, key, mode):
        c_map = {
            ('B', 'major'): '01B', ('F#', 'major'): '02B', ('C#', 'major'): '03B', ('G#', 'major'): '04B',
            ('D#', 'major'): '05B', ('A#', 'major'): '06B', ('F', 'major'): '07B', ('C', 'major'): '08B',
            ('G', 'major'): '09B', ('D', 'major'): '10B', ('A', 'major'): '11B', ('E', 'major'): '12B',
            ('G#', 'minor'): '01A', ('D#', 'minor'): '02A', ('A#', 'minor'): '03A', ('F', 'minor'): '04A',
            ('C', 'minor'): '05A', ('G', 'minor'): '06A', ('D', 'minor'): '07A', ('A', 'minor'): '08A',
            ('E', 'minor'): '09A', ('B', 'minor'): '10A', ('F#', 'minor'): '11A', ('C#', 'minor'): '12A'
        }
        return c_map.get((key, mode.lower()), "00X")

    def process_batch(self, batch_data):
        paths = [x[0] for x in batch_data]
        audios = [x[1] for x in batch_data]
        durations = [x[2] for x in batch_data]
        dsp_tensor = torch.zeros(len(audios), self.sr * 120, device=self.device)
        for i, a in enumerate(audios):
            l = min(len(a), self.sr * 120)
            dsp_tensor[i, :l] = torch.from_numpy(a[:l]).to(self.device)
        
        with torch.no_grad():
            spec = self.cqt_layer(dsp_tensor)
            energy = spec.pow(2).mean(dim=(1, 2)).cpu().numpy()
            chroma = spec.view(len(audios), 7, 12, -1).sum(dim=(1, 3))
            chroma = chroma / (chroma.norm(dim=1, keepdim=True) + 1e-6)
            corrs = torch.matmul(chroma, self.profiles)
            best_idx = torch.argmax(corrs, dim=1).cpu().numpy()
            
            embeddings_list = []
            pc = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
            for i in range(len(audios)):
                mid = len(audios[i])//2
                audio_slice = audios[i][mid : mid + 22050*15] if len(audios[i]) > 22050*15 else audios[i]
                input_values = self.processor(audio_slice, sampling_rate=22050, return_tensors="pt").input_values.to(self.device)
                emb = self.mert_model(input_values).last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
                embeddings_list.append(emb.tolist())
                
        results = []
        for i in range(len(audios)):
            res = {
                'key': pc[best_idx[i] % 12], 'mode': 'major' if best_idx[i] < 12 else 'minor',
                'energy': float(energy[i]), 'duration': float(durations[i]), 
                'embedding': embeddings_list[i], 'path': paths[i]
            }
            try:
                onset = librosa.onset.onset_strength(y=audios[i][:22050*60], sr=22050)
                res['bpm'] = float(librosa.beat.tempo(onset_envelope=onset, sr=22050, aggregate=np.mean)[0])
            except: res['bpm'] = 120.0
            res['camelot'] = self.get_camelot(res['key'], res['mode'])
            results.append(res)
        return results

### 3. 💎 Spectre Restoration Unit (High-End Reconstruction)

In [ ]:
class SpectreRestorer:
    """
    DSP Restoration Unit: 
    Upsampling -> Spectral Gap Detection -> Harmonics Synthesis -> FLAC Export.
    """
    def __init__(self, target_sr=48000, device='cuda'):
        self.target_sr = target_sr
        self.device = device

    def detect_cutoff(self, waveform, sr):
        # Perform STFT to find the MP3 frequency shelf
        stft = torch.stft(waveform, n_fft=2048, hop_length=512, window=torch.hann_window(2048).to(self.device), return_complex=True)
        mag = torch.abs(stft).mean(dim=-1).mean(dim=0)
        mag_db = 20 * torch.log10(mag + 1e-8)
        max_db = torch.max(mag_db)
        freqs = torch.linspace(0, sr/2, len(mag_db))
        
        # Find where energy drops below -55dB relative to peak
        threshold = max_db - 55
        mask = mag_db > threshold
        if not mask.any(): return 16000.0
        cutoff = freqs[mask][-1].item()
        return max(12000.0, min(cutoff, 20000.0))

    def restore(self, input_path, output_path):
        try:
            waveform, sr = torchaudio.load(input_path)
            waveform = waveform.to(self.device)
            
            # 1. Upsampling to 48kHz
            if sr != self.target_sr:
                resampler = T.Resample(sr, self.target_sr).to(self.device)
                waveform = resampler(waveform)
            
            # 2. Analyze Magnitude for Cutoff
            cutoff = self.detect_cutoff(waveform[0], self.target_sr)
            
            # 3. High-Pass Isolation
            # We extract the 'air' band just below the cutoff to generate harmonics
            y_high = F.highpass_biquad(waveform, self.target_sr, cutoff * 0.85)
            
            # 4. Non-linear Harmonic Synthesis (Saturation)
            # Creating new harmonics using a tanh curve for pleasant 'Spectre' style excitement
            gain = 2.5 
            y_excited = torch.tanh(y_high * gain) 
            
            # Apply a steep HPF to the excited signal so it only fills the GAP
            y_excited = F.highpass_biquad(y_excited, self.target_sr, cutoff)
            
            # 5. Parallel Mix (12% Wet for subtle air restoration)
            wet_level = 0.12
            y_final = waveform + (y_excited * wet_level)
            
            # Normalize to avoid clipping
            y_final = y_final / (torch.max(torch.abs(y_final)) + 1e-6)
            
            # 6. Export as FLAC Level 5
            torchaudio.save(output_path, y_final.cpu(), self.target_sr, encoding="PCM_S", bits_per_sample=16)
            return True
        except Exception as e:
            print(f"⚠️ Restoration failed for {input_path}: {e}")
            return False

### 4. 🌈 Chromatic Mixer & Affinity Logic

In [ ]:
def get_next_harmonic(current_cam):
    num, alpha = int(current_cam[:2]), current_cam[2]
    return [
        f"{str(num).zfill(2)}{alpha}",               # Same
        f"{str((num % 12) + 1).zfill(2)}{alpha}",    # +1 Chromatic
        f"{str(((num - 2) % 12) + 1).zfill(2)}{alpha}", # -1 Chromatic
        f"{str(num).zfill(2)}{'A' if alpha == 'B' else 'B'}" # Parallel
    ]

def sequence_chromatic_set(tracks, target_duration):
    if not tracks: return []
    pool = list(tracks)
    current = pool.pop(0)
    ordered_set = [current]
    current_dur = current['duration']
    
    while pool and current_dur < target_duration:
        compat_keys = get_next_harmonic(current['camelot'])
        def score(t):
            h_score = 1.0 if t['camelot'] in compat_keys else (0.8 if t['camelot'] == current['camelot'] else 0.0)
            bpm_diff = abs(t['bpm'] - current['bpm'])
            b_score = max(0, 1.0 - (bpm_diff / 40.0))
            s_score = np.dot(current['embedding'], t['embedding']) / (np.linalg.norm(current['embedding']) * np.linalg.norm(t['embedding']) + 1e-9)
            return (h_score * 0.5) + (s_score * 0.3) + (b_score * 0.2)
        
        pool.sort(key=score, reverse=True)
        next_t = pool.pop(0)
        ordered_set.append(next_t)
        current_dur += next_t['duration']
        current = next_t
    return ordered_set

def clean_name(n):
    n = os.path.basename(n).rsplit('.', 1)[0]
    n = re.sub(r"^[\w\-]+?-", "", n)
    return re.sub(r"[\-\_\.]+?", " ", n).strip()

### 4. 🚀 Execution Pipeline

In [ ]:
# CONFIG
INPUT_DIR = "/kaggle/input/datasets/danieldobles/slavic-set"
OUTPUT_DIR = "/kaggle/working/master_organized"
BATCH_SIZE = 16
SET_DUR = 75 * 60 
N_CLUSTERS = 3
ENABLE_RESTORATION = True # Set to False to just copy files without Spectre Enhancements

master = SpectralMasterEngine(device=device)
restorer = SpectreRestorer(device=device)

print("🔍 Scanning Library...")
paths = []
for ext in ['*.mp3', '*.wav', '*.flac', '*.m4a']:
    paths.extend(glob.glob(os.path.join(INPUT_DIR, "**", ext), recursive=True))
paths = list(set(paths))

to_analyze = [p for p in paths if p not in master.cache]
if to_analyze:
    chunks = [to_analyze[i:i+BATCH_SIZE] for i in range(0, len(to_analyze), BATCH_SIZE)]
    with ThreadPoolExecutor(max_workers=2) as pool:
        for chunk in tqdm(chunks, desc="🔥 AI Analysis"):
            def load(p):
                try: y, _ = librosa.load(p, sr=22050); return (p, y, len(y)/22050)
                except: return None
            batch_data = [x for x in list(pool.map(load, chunk)) if x is not None]
            results = master.process_batch(batch_data)
            for r in results: master.cache[r['path']] = r
            master.save_cache(); gc.collect()

library = [master.cache[p] for p in paths if p in master.cache]
X = normalize(np.array([t['embedding'] for t in library]))
p_labels = (cuKMeans(n_clusters=N_CLUSTERS) if HAS_CUML else skKMeans(n_clusters=N_CLUSTERS, n_init=10)).fit_predict(X)

clusters = {i: [] for i in range(N_CLUSTERS)}
for i, l in enumerate(p_labels): clusters[l].append(library[i])

if os.path.exists(OUTPUT_DIR): shutil.rmtree(OUTPUT_DIR)
print("🌈 Sequencing & Processing Sets...")
total_sets = 0
for c_idx, cluster_tracks in clusters.items():
    c_name = f"Group_{chr(65+c_idx)}"
    temp_pool = sorted(cluster_tracks, key=lambda x: x['energy'])
    set_idx = 1
    while temp_pool:
        ordered_set = sequence_chromatic_set(temp_pool, SET_DUR)
        if not ordered_set: break
        temp_pool = [t for t in temp_pool if t['path'] not in {s['path'] for s in ordered_set}]
        
        s_dir = os.path.join(OUTPUT_DIR, c_name, f"Set_{set_idx}")
        os.makedirs(s_dir, exist_ok=True)
        
        for i, t in enumerate(tqdm(ordered_set, desc=f"💎 {c_name} Set {set_idx}", leave=False)):
            meta = f"[{t['camelot']} - {int(t['bpm'])}BPM]"
            ext = "flac" if ENABLE_RESTORATION else t['path'].rsplit('.', 1)[1]
            out_name = f"{str(i+1).zfill(2)} - {meta} {clean_name(t['path'])}.{ext}"
            out_path = os.path.join(s_dir, out_name)
            
            if ENABLE_RESTORATION:
                restorer.restore(t['path'], out_path)
            else:
                shutil.copy2(t['path'], out_path)
        
        set_idx += 1; total_sets += 1

print(f"✅ SUCCESS! Generated {total_sets} high-fidelity sets.")
zip_name = "SpectralAffinity_Restored_MasterMix.zip"
!zip -0 -rq {zip_name} master_organized
display(HTML(f"<h3>🚀 <a href='{zip_name}' id='dl'>DOWNLOAD SPECTRE MASTER MIXES</a></h3>"))
display(HTML("<script>setTimeout(() => document.getElementById('dl').click(), 1000);</script>"))